### Basic ReAct Agent using Langgraph

In [1]:
# !pip install langgraph

### Prompts

In [2]:
#system prompt
# sys = """
# You are an expert academic professor specializing in explaining complex concepts in a simple and engaging manner. You don't just answer directly but you break whole solution into multple parts and then create questions for each part to help the user understand the concept step by step. You also provide examples and analogies to make the explanation relatable. Your goal is to ensure that the user not only understands the concept but can also apply it in real-world scenarios.
# Your task includes:
# 1. Breaking down complex concepts into manageable small parts.
# 2. Creating questions for each part to guide the user through the learning process.
# 3. If user gets answer wrong for small part, you will provide hints and ask them to try again.
# 4. If user gets answer wrong twice, you will provide the correct answer and explain why it is correct.
# 5. If user gets answer right, you appreciate them and ask next question.
# 6. You follow this cycle until whole explanation is complete.
# """

sys = """
Your name is AcadGenie, an expert academic assistant specializing in guiding learners through complex concepts by breaking them down into manageable steps. Your role is not only to provide answers, but to foster deep understanding through an interactive dialogue. 
You use multiple diagnostic steps to ensure clarity and comprehension.

You follow a guided practice flow, structured as follows:
1. When a user asks a question, you first evaluate whether the question should be broken down into multiple smaller diagnostic steps.
   - If it is simple, you provide a direct but clear answer with brief context or example.
   - If it is complex, you break it into smaller parts and guide the user through each.

2. For complex questions:
   - Decompose the concept into smaller steps.
   - Create one guiding question per step.
   - Ask each question one by one.

3. For each guiding question:
   - If the user answers correctly, affirm and proceed to the next step.
   - If the user answers incorrectly:
     a. Give a hint on the first wrong attempt.
     b. Give the correct answer and an explanation on the second wrong attempt.
   - Reinforce understanding with examples or analogies.

4. After the full concept has been explored:
   - Ask if the user wants to explore another question.
   - If they do, repeat the above cycle.

Your tone is warm, supportive, and highly pedagogical. Always aim to ensure the user gains true conceptual clarity and can apply their knowledge in real-world or academic contexts.

## Output Format:
Each question you create should always be a multiple-choice question but it could be true/false MCQ, fill in the blank MCQ, etc.
Each question should be in following format:
```json
{
    "question": "Your question here?",
    "options": [
        {"option": "A", "text": "Option A text", "DR": "misconception or common mistake which may lead to this answer"},
        {"option": "B", "text": "Option B text", "DR": "misconception or common mistake which may lead to this answer"},
        {"option": "C", "text": "Option C text", "DR": "misconception or common mistake which may lead to this answer"},
        {"option": "D", "text": "Option D text", "DR": "misconception or common mistake which may lead to this answer"}
    ],
    "correct_option": "A"  # or B, C, D depending on the correct answer,
    "explanation": "A brief explanation of why the correct answer is correct and why the others are not.",
    "comment": "A brief comment to encourage the user or provide additional context. For example, 'Great job! This concept is crucial for understanding X.', 'This is a common misconception. Let's clarify it.', etc."
}
```
"""


In [3]:
from pydantic import BaseModel, Field
from typing import List, Dict

class Option(BaseModel):
    option: str
    text: str
    DR: str

class step_question_response(BaseModel):
    question: str
    options: List[Option]
    correct_option: str
    explanation: str
    comment: str
    hint: str
    
    class Config:
        validate_by_name = True

In [4]:
# from langchain.chat_models import init_chat_model
# from langgraph.prebuilt import create_react_agent
# from langgraph.checkpoint.memory import InMemorySaver

# checkpointer = InMemorySaver()

# model = init_chat_model(
#     "openai:gpt-4o",
#     temperature=0.3
# )

# agent = create_react_agent(
#     model=model,
#     prompt=sys,
#     checkpointer=checkpointer,
#     response_model=step_question_response,
#     # max_iterations=10,
#     # max_steps=5,
# )

# config = {"configurable": {"thread_id": "1"}}
# sf_response = agent.invoke(
#     {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
#     config  
# )
# ny_response = agent.invoke(
#     {"messages": [{"role": "user", "content": "what about new york?"}]},
#     config
# )

In [5]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain.schema import HumanMessage, AIMessage

# Create a conversation history class
class ConversationHistory:
    def __init__(self):
        self.messages = []

    def add_message(self, role: str, content: str):
        self.messages.append({"role": role, "content": content})

    def get_messages(self):
        return self.messages

# Define the dynamic prompt for the agent
def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"{sys}. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

# Initialize conversation history
# conversation = ConversationHistory()
checkpointer = InMemorySaver()

model = init_chat_model(
    "openai:gpt-4o",
    temperature=0.3
)

agent = create_react_agent(
    model=model,
    prompt=prompt,
    tools=[],
    checkpointer=checkpointer,
    response_format={
        "name": "step_question_response",
        "description": "Response format for educational questions",
        "schema": step_question_response.model_json_schema()
    }
)

config = {"configurable": {"user_name": "Shishir Dwivedi", "thread_id": "1"}}

# def get_response(user_input: str) -> dict:
#     # Add user message to history
#     conversation.add_message("user", user_input)
    
#     # Get response using full conversation history
#     response = agent.invoke(
#         {"messages": conversation.get_messages()},
#         config
#     )
#     print("Agent: ", response)
#     # Add assistant's response to history
#     conversation.add_message("assistant", response.dict())
    
#     return response

# responses = []

# while True:
#     user_input = input("You: ")
#     if user_input.lower() == "exit":
#         break
#     response = get_response(user_input)
#     print("Assistant:", response)

def get_response(user_input: str) -> dict:
    # Add user message to history
    # conversation.add_message("user", user_input)
    
    # Get response using full conversation history
    print("\n--- Conversation History ---")
    # for msg in conversation.get_messages():
    #     print(f"{msg['role'].capitalize()}: {msg['content']}")
    print("-----------------------------")
    print("\n--- Asking Agent ---")
    print("User:", user_input)
    # Invoke the agent with the conversation history
    response = agent.invoke(
        {"messages": [{"role": "user", "content": user_input}]},
        config
    )
    
    # Extract messages from response
    messages = response.get('messages', [])
    human_message = next((msg for msg in messages if isinstance(msg, HumanMessage)), None)
    ai_message = next((msg for msg in messages if isinstance(msg, AIMessage)), None)
    
    # Extract structured response
    structured_response = response.get('structured_response', {})
    
    # if ai_message:
        # conversation.add_message("assistant", structured_response)
    
    if human_message:
        print("\nHuman:", human_message.content)
    if ai_message:
        print("Assistant:", ai_message.content)
    
    if structured_response:
        print("\n--- Structured Response ---")
        print("Question:", structured_response['question'])
        print("\nOptions:")
        for option in structured_response['options']:
            print(f"{option['option']}: {option['text']}")
        print("\nCorrect Answer:", structured_response['correct_option'])
        print("\nExplanation:", structured_response['explanation'])
        print("\nComment:", structured_response['comment'])
        print("-------------------------")
    
    return {
        'human_message': human_message.content if human_message else None,
        'ai_message': ai_message.content if ai_message else None,
        'structured_response': structured_response
    }

while True:
    user_input = input("\nYou: ")
    if user_input.lower() == "exit":
        break
    response = get_response(user_input)
    # ai_response = response['ai_message']
    # structured_data = response['structured_response']

In [6]:
# import streamlit as st
# from react_agent_langgraph import get_response, conversation  # Import from your notebook file

# def init_session_state():
#     if "messages" not in st.session_state:
#         st.session_state.messages = []

# def display_message(role, content, structured_data=None):
#     with st.chat_message(role):
#         st.write(content)
#         if structured_data:
#             with st.expander("Question Details"):
#                 st.write("**Question:**", structured_data['question'])
#                 st.write("**Options:**")
#                 for option in structured_data['options']:
#                     st.write(f"- {option['option']}: {option['text']}")
                
#                 # Create columns for answer-related information
#                 col1, col2 = st.columns(2)
#                 with col1:
#                     st.write("**Correct Answer:**", structured_data['correct_option'])
#                 with col2:
#                     if st.button("Show Explanation", key=f"explain_{len(st.session_state.messages)}"):
#                         st.write("**Explanation:**", structured_data['explanation'])
#                         st.write("**Comment:**", structured_data['comment'])

# def main():
#     st.title("Educational AI Assistant")
#     st.write("""
#     Welcome to your interactive learning session! 
#     Ask any question, and I'll guide you through the concept step by step.
#     """)
    
#     init_session_state()
    
#     # Display chat history
#     for message in st.session_state.messages:
#         display_message(
#             message["role"],
#             message["content"],
#             message.get("structured_data")
#         )
    
#     # Chat input
#     if prompt := st.chat_input("What would you like to learn about?"):
#         # Display user message
#         display_message("user", prompt)
#         st.session_state.messages.append({"role": "user", "content": prompt})
        
#         # Get AI response
#         response = get_response(prompt)
        
#         # Display AI response
#         display_message(
#             "assistant",
#             response['ai_message'] if response['ai_message'] else "",
#             response['structured_response']
#         )
        
#         # Save to session state
#         st.session_state.messages.append({
#             "role": "assistant",
#             "content": response['ai_message'] if response['ai_message'] else "",
#             "structured_data": response['structured_response']
#         })

# if __name__ == "__main__":
#     main()

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class NotebookChatUI:
    def __init__(self):
        self.messages = []
        self.output = widgets.Output()
        self.text_input = widgets.Text(
            placeholder='Ask a question...',
            description='You:',
            layout=widgets.Layout(width='80%')
        )
        self.send_button = widgets.Button(description='Send')
        self.send_button.on_click(self.on_send)
        
        # Layout
        self.container = widgets.VBox([
            self.output,
            widgets.HBox([self.text_input, self.send_button])
        ])
    
    def display_message(self, role, content, structured_data=None):
        with self.output:
            print(f"{role.title()}: {content}")
            if structured_data:
                print("\nQuestion Details:")
                print(f"Q: {structured_data['question']}")
                print("\nOptions:")
                for option in structured_data['options']:
                    print(f"- {option['option']}: {option['text']}")
    
    def on_send(self, button):
        user_input = self.text_input.value
        self.text_input.value = ''
        
        if user_input.lower() == 'exit':
            return
        
        # Display user message
        self.display_message('user', user_input)
        
        # Get and display AI response
        response = get_response(user_input)
        self.display_message(
            'assistant', 
            response['ai_message'] if response['ai_message'] else "",
            response['structured_response']
        )
    
    def start(self):
        display(self.container)

# Create and start the chat UI
chat_ui = NotebookChatUI()
chat_ui.start()